# Question 1:
## 1.1. Notebook book created
import the basic dependencies. 

In [1]:
import numpy as np 
import pandas as pd
import requests # Library for web scraping

print('Libraries imported.')

Libraries imported.


## 1.2. Web page scraped
About the Data, Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.
   - is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.
   - Scraping table from HTML using BeautifulSoup

In [2]:
# To run this, you can install BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# Or download the file
# http://beautiful-soup-4
# and unzip it in the same directory as this file
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [3]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [7]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
#soup ready
table = soup.find('table',{'class':'wikitable sortable'})
#table
table_rows = table.find_all('tr')
#table_rows
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])


## 1.3. Data transformed into pandas dataframe

In [10]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode       180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


In [9]:
df.shape

(180, 3)

## 1.4. Dataframe cleaned and notebook annotate
Only process the cells that have an assigned borough, we can ignore cells with 'Not assigned' boroughs, like in rows 1 & 2

In [11]:
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df1 = df.reset_index()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index            103 non-null int64
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [12]:
df1.shape

(103, 4)

In [15]:
df1.head()

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


More than one neighborhood can exist in one postal code area, M5A is listed twice and has two neighborhoods Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma using **groupby**

In [13]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [16]:
df2.shape

(103, 2)

There are also cells that have an assigned neighbouhoods,like M7A, lets assign their boroughs as their neighbourhood, as follows:

In [96]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()

Now we can remove the duplicate boroughts as follows:

In [97]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [98]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [99]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [100]:
df3.shape

(103, 3)

# Question 2:
## Used the Geocoder Package to get the coordinates of neighborhoods

In [76]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")

In [114]:
df4=df3.drop('Neighbourhood', axis=1).join(df3['Neighbourhood'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Neighbourhood'))
df4=df4.reset_index(drop=True)

In [115]:
df4['Address']=df4['PostalCode'] + ',' + df4['Borough'] + ','+ df4['Neighbourhood']
df4['Coordinates'] =df4['Address'].apply(geolocator.geocode)
df4.head()

,PostalCode,Borough,Neighbourhood,Address,Coordinates
0,M1B,Scarborough,Malvern,"M1B,Scarborough,Malvern",None
1,M1B,Scarborough,Rouge,"M1B,Scarborough, Rouge","(Rouge, Scarborough—Rouge Park, Scarborough, T..."
2,M1C,Scarborough,Rouge Hill,"M1C,Scarborough,Rouge Hill","(Rouge Hill, Scarborough—Rouge Park, Scarborou..."
3,M1C,Scarborough,Port Union,"M1C,Scarborough, Port Union","(Port Union, Scarborough—Rouge Park, Scarborou..."
4,M1C,Scarborough,Highland Creek,"M1C,Scarborough, Highland Creek","(Highland Creek, Scarborough—Rouge Park, Scarb..."


In [117]:
df4['Latitude']=df4['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df4['Longitude']=df4['Coordinates'].apply(lambda x: x.longitude if x !=None else None)

In [119]:
columnlist = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
df4=df4[columnlist]
df4

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,NaN,NaN
1,M1B,Scarborough,Rouge,43.804930,-79.165837
2,M1C,Scarborough,Rouge Hill,43.795019,-79.135104
3,M1C,Scarborough,Port Union,43.775504,-79.134976
4,M1C,Scarborough,Highland Creek,43.790117,-79.173334
...,...,...,...,...,...
204,M9V,Etobicoke,Mount Olive,NaN,NaN
205,M9V,Etobicoke,Beaumond Heights,NaN,NaN
206,M9V,Etobicoke,Thistletown,NaN,NaN
207,M9V,Etobicoke,Albion Gardens,NaN,NaN


In [125]:
Neighbourhoods=df4[df4['Latitude'].notna()].groupby('PostalCode')
Neighbourhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
1,M1B,Scarborough,Rouge,43.804930,-79.165837
2,M1C,Scarborough,Rouge Hill,43.795019,-79.135104
3,M1C,Scarborough,Port Union,43.775504,-79.134976
4,M1C,Scarborough,Highland Creek,43.790117,-79.173334
8,M1G,Scarborough,Woburn,43.759824,-79.225291
35,M1W,Scarborough,Steeles West,43.816178,-79.314538
36,M1W,Scarborough,L'Amoreaux West,43.799003,-79.305967
148,M6N,York,Runnymede,43.665580,-79.482108
149,M6N,York,The Junction North,42.928672,-78.026122
187,M9C,Etobicoke,Eringate,43.662273,-79.576516
